In [1]:
import torch
import torch.nn as nn
import torch.functional as F 
from tqdm import tqdm
from model.mscred import MSCRED
from utils.data import load_data
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
def train(dataLoader, model, optimizer, epochs, device):
    model = model.to(device)
    print("------training on {}-------".format(device))
    for epoch in range(epochs):
        train_l_sum,n = 0.0, 0
        for x in tqdm(dataLoader):
            x = x.to(device)
            x = x.squeeze()
            #print(type(x))
            l = torch.mean((model(x)-x[-1].unsqueeze(0))**2)
            train_l_sum += l
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            n += 1
            #print("[Epoch %d/%d][Batch %d/%d] [loss: %f]" % (epoch+1, epochs, n, len(dataLoader), l.item()))
            
        print("[Epoch %d/%d] [loss: %f]" % (epoch+1, epochs, train_l_sum/n))

def test(dataLoader, model):
    print("------Testing-------")
    index = 800
    loss_list = []
    reconstructed_data_path = "./utils/data/matrix_data/reconstructed_data/"
    
    if not os.path.exists(reconstructed_data_path):
        os.makedirs(reconstructed_data_path)
    
    with torch.no_grad():
        for x in dataLoader:
            x = x.to(device)
            x = x.squeeze()
            reconstructed_matrix = model(x) 
            path_temp = os.path.join(reconstructed_data_path, 'reconstructed_data_' + str(index) + ".npy")
            np.save(path_temp, reconstructed_matrix.cpu().detach().numpy())
            # l = criterion(reconstructed_matrix, x[-1].unsqueeze(0)).mean()
            # loss_list.append(l)
            # print("[test_index %d] [loss: %f]" % (index, l.item()))
            index += 1

In [3]:
if __name__ == '__main__':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("device is", device)
    dataLoader = load_data()
    mscred = MSCRED(3, 256)

    # 训练阶段
    # mscred.load_state_dict(torch.load("./checkpoints/model1.pth"))
    optimizer = torch.optim.Adam(mscred.parameters(), lr = 0.0002)
    train(dataLoader["train"], mscred, optimizer, 10, device)

device is cuda
------training on cuda-------


  0%|                                                                                           | 0/789 [00:00<?, ?it/s]

input.shape:  torch.Size([5, 32, 30, 30])
x.shape:  torch.Size([1, 32, 30, 30])


IndexError: index 1 is out of bounds for dimension 0 with size 1

In [ ]:
    print("保存模型中....")
    #torch.save(mscred.state_dict(), "./checkpoints/model2.pth")

In [ ]:
    # # 测试阶段
    mscred.load_state_dict(torch.load("./checkpoints/model2.pth"))
    mscred.to(device)
    test(dataLoader["test"], mscred)

In [ ]:
    sum = 0
    #criterion = torch.nn.MSELoss()
    criterion = torch.sub
    
    print("------Testing-------")
    index = 800
    loss_list = []
    reconstructed_data_path = "./utils/data/matrix_data/reconstructed_data/"
    
    if not os.path.exists(reconstructed_data_path):
        os.makedirs(reconstructed_data_path)
    
    with torch.no_grad():
        for x in dataLoader['test']:
            x = x.to(device)
            x = x.squeeze()
            reconstructed_matrix = mscred(x) 
            #path_temp = os.path.join(reconstructed_data_path, 'reconstructed_data_' + str(index) + ".npy")
            #np.save(path_temp, reconstructed_matrix.cpu().detach().numpy())
            l = criterion(reconstructed_matrix, x[-1].unsqueeze(0)).mean()
            sum += l
            loss_list.append(l)
            #print("[test_index %d] [loss: %f]" % (index, l.item()))
            index += 1

In [ ]:
mean = sum / len(loss_list)

In [ ]:
mean

In [ ]:
torch.stack(loss_list[:10])[:]

In [ ]:
std = torch.std(torch.stack(loss_list[:]))

In [ ]:
std

In [ ]:
mean + std * 2